In [10]:
import pandas as pd

# do for 10 shows
from tqdm import tqdm
import time
import difflib
import json



In [ ]:

pods = pd.read_csv("./0_pods.csv")

print("total pods: ", len(pods))
pods = pods[
    pods["categories"].notna() 
    # pods["categories"].apply(lambda x: len(x) if isinstance(x, (list, str)) else 0) > 0
]
pods.head()
print("total pods after removing null categories: ", len(pods))


pods = pods[
    pods["itunesID"].notna() &
    pods["youtubeID"].notna()
]
pods = pods[["id", "name","itunesID", "image","youtubeID", "categories", "summary", "spotifyId","socialLinks"]]
print("total pods after removing null itunesID and youtubeID: ", len(pods))
pods.head()


pods["category"] = None
pods["category"] = pods["category"].astype('str')


pods["apple_reviews"] = None
pods["apple_reviews"] = pods["apple_reviews"].astype('object')


pods["followers_count"] = {}
pods["followers_count"] = pods["followers_count"].astype('object')

pods["recent_posts"] = {}
pods["recent_posts"] = pods["recent_posts"].astype('object')


pods["youtube_videos"] = None
pods["youtube_videos"] = pods["youtube_videos"].astype('object')


total pods:  450
total pods after removing null categories:  376
total pods after removing null itunesID and youtubeID:  275


In [22]:
transcripts = {}

def save_transcripts_to_file():
    """Save all transcripts to JSON file"""
    with open("video-transcripts.json", "w") as f:
        json.dump(transcripts, f, indent=2)

def load_transcripts_from_file():
    """Load transcripts from JSON file"""
    global transcripts
    try:
        with open("video-transcripts.json", "r") as f:
            transcripts = json.load(f)
    except FileNotFoundError:
        transcripts = {}

def store_transcript(video_id, transcript_text):
    """Store transcript for a video"""
    transcripts[video_id] = transcript_text
    save_transcripts_to_file()

def get_transcript(video_id):
    """Get transcript by video ID"""
    return transcripts.get(video_id, "")

load_transcripts_from_file()



In [12]:
stoic_base_url = "http://127.0.0.1:5000"

In [13]:

import re

def get_instagram_username(url):
    """
    Extract Instagram username from URL - simple and reliable
    
    Args:
        url (str): Instagram URL
        
    Returns:
        str: Username or None if not found
    """
    if not url:
        return None
    
    # Remove any whitespace
    url = url.strip()
    
    # Use regex to extract username from Instagram URL
    # This pattern captures the username part after instagram.com/
    pattern = r'instagram\.com/([^/?#]+)'
    
    match = re.search(pattern, url, re.IGNORECASE)
    
    if match:
        username = match.group(1)
        return username
    
    return None

In [14]:
import requests
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

def get_pod_apple_reviews(itunes_id):
    apple_reviews_api_url = f"{stoic_base_url}/api/applepodcast/reviews?itunes_id={itunes_id}"
    response = requests.get(apple_reviews_api_url)
    apple_reviews_data = response.json()
    return apple_reviews_data


def get_pod_social_followers(social_links):
    if isinstance(social_links, str):
        social_links = json.loads(social_links)

    followers_count = {}
    recent_posts = {}
    try:
        instagram_url = social_links.get("instagram")
        instgaram_username = get_instagram_username(instagram_url)

        if instgaram_username:
            instagram_api_url = f"{stoic_base_url}/api/instagram/profile?username={instgaram_username}"
            response = requests.get(instagram_api_url)
            instagram_data = response.json()

            recent_posts["instagram"] = instagram_data.get("recentPosts", [])
            if "followersCount" in instagram_data:
                followers_count["instagram"] = instagram_data["followersCount"]
    except Exception as e:
        print("error in instagram: ",e)


    # try:
    #     twitter_url = social_links.get("twitter") or social_links.get("x")
    #     twitter_username = twitter_url.split("/")[-1]
    #     twitter_api_url = f"{stoic_base_url}/api/twitter/profile?username={twitter_username}"
    #     print("twitter api url: ",twitter_api_url)
    #     response = requests.get(twitter_api_url)
    #     twitter_data = response.json()
    #     recent_posts["twitter"] = []
    #     if "followersCount" in twitter_data:
    #         followers_count["twitter"] = twitter_data["followersCount"]
    # except Exception as e:
    #     print("error in twitter: ",e)

    return followers_count, recent_posts

def fetch_single_video(youtube_video_api_url, video_id):
    """Fetch details for a single video"""
    try:
        payload = {"video_id": video_id, "fetch_transcript": True}
        response = requests.post(youtube_video_api_url, json=payload)
        youtube_video_data = response.json()
        return video_id, youtube_video_data
    except Exception as e:
        print(f"Error fetching video {video_id}: {e}")
        return video_id, None

def get_youtube_info_and_latest_videos(youtube_id, video_count=5, max_workers=2):
    youtube_latest_videos_api_url = f"{stoic_base_url}/api/youtube/get-channel-details-and-latest-videos"
    payload = {"channel_identifier": youtube_id, "latest_video_count":video_count}
    response = requests.post(youtube_latest_videos_api_url, json=payload)
    youtube_data = response.json()
    subscribers = None
    if "subscriberCount" in youtube_data.get("channelDetails", {}):
        subscribers = youtube_data["channelDetails"]["subscriberCount"]
    latest_basic_videos = youtube_data["latestVideos"]

    youtube_video_api_url = f"{stoic_base_url}/api/youtube/video"
    latest_videos = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        future_to_video_id = {
            executor.submit(fetch_single_video, youtube_video_api_url, video["youtubeVideoId"]): video["youtubeVideoId"]
            for video in latest_basic_videos
        }
        
        # Collect results as they complete
        for future in as_completed(future_to_video_id):
            video_id, result = future.result()
            if result is not None:
                latest_videos.append(result)


    return latest_videos, subscribers

In [ ]:
from openai import OpenAI
import json
import os

os.environ["OPENAI_API_KEY"] = \n

client = OpenAI()

In [16]:
def get_main_category(categories, podcast_name, podcast_summary):
    categories_str = ", ".join(categories)
    prompt = f"""
    You are given a list of categories: {categories_str}.
    The podcast name is: "{podcast_name}".
    The summary is: "{podcast_summary}".

    Select the **single most relevant category** from the given categories 
    that best describes this podcast. Return only the category name. return the category as it is. dont add space or any other character. and everything should in lowercase
    """

    response = client.responses.create(
        model="gpt-4.1",
        input=prompt,
        temperature=0
    )
    return response.output_text.strip()

In [17]:
def prepare_transcript(transcript, max_chars=400000):
    """
    Combine transcript into a single string with timestamps.
    Truncate if exceeds max_chars.
    """
    combined = []
    total_length = 0

    for item in transcript:
        text_piece = f"[{item['start']}s] {item['text']} "
        if total_length + len(text_piece) > max_chars:
            break
        combined.append(text_piece)
        total_length += len(text_piece)
    
    return "".join(combined)

def extract_sponsors_from_transcript(transcripts, description):
    """
    Extract sponsors from full transcript (up to max_chars) and description.
    """

    transcript_text= prepare_transcript(transcripts)

    prompt = f"""
    You are given a podcast transcript and YouTube video description.
    Extract all **real sponsors** (products or services, not social media links),
    along with the start and end timestamps, in this JSON format:
    [
      {{
        "sponsorName": "",
        "startTime": 0,
        "endTime": 0,
        "domain": ""
      }}
    ]

    **Transcript:** {transcript_text}
    **Description:** {description}

    Guidelines:
    1. If no promotions are found, return an empty array [].
    2. Ignore social links like Spotify, iTunes, YouTube, Instagram, Facebook, TikTok, Patreon, Twitch.
    3. Use timestamps from the transcript when possible.
    4. Extract domain names from the description links.
    5. Return ONLY valid JSON.
    """

    response = client.responses.create(
        model="gpt-4o-mini",
        # model="gpt-4.1",
    # model: "gpt-4o-mini",
        input=prompt,
        temperature=0
    )

    raw_output = response.output_text.strip()

    # Remove code fences like ```json and ```
    if raw_output.startswith("```"):
        raw_output = raw_output.strip("`")
        raw_output = raw_output.replace("json\n", "").replace("json", "")

    try:
        sponsors = json.loads(raw_output)
    except Exception as e:
        print("Exception: ", e)
        sponsors = []

    return sponsors

In [18]:

with open("./categories_details.json", "r") as file:
    data = json.load(file)
all_categories = data["all_categories"]
category_rules = data["category_rules"]
avg_ad_price = data["avg_ad_price"]

def match_category(input_category: str) -> str:
    input_category = input_category.lower().strip()
    for keyword, mapped_category in category_rules.items():
        if keyword in input_category:
            return mapped_category
    closest_match = difflib.get_close_matches(input_category, all_categories, n=1, cutoff=0.4)
    return closest_match[0] if closest_match else "unknown"



matched = match_category("crime")
print(matched)

true_crime


In [19]:
def get_pod_all_info(pod, index):
    # get information about podcast
    pod_id = pod["id"]
    pod_itunes_id = int(pod["itunesID"])
    pod_name = pod["name"]
    pod_summary = pod["summary"]
    pod_categories = pod["categories"]

    existing_category = pod["category"]
    if existing_category is None or existing_category == "" or existing_category == "None":
        try:
            pod_category = get_main_category(pod_categories, pod_name, pod_summary)
            pods.at[index, "category"] = pod_category
        except Exception as e:
            print(pod_id," error in getting category: ",e)

    existing_apple_reviews = pod["apple_reviews"]
    if existing_apple_reviews is None:
        try:
            apple_reviews_data = get_pod_apple_reviews(pod_itunes_id)
            pods.at[index, "apple_reviews"] = {
                "average_rating": apple_reviews_data["average_rating"],
                "rating_count": apple_reviews_data["rating_count"],
                    "ratings_breakdown": apple_reviews_data["ratings_breakdown"],
                }
        except Exception as e:
            print(pod_id," error in getting apple reviews: ",e)


    pod_social_links = pod["socialLinks"]
    existing_followers = pod["followers_count"]
    try:
        existing_insta_followers = existing_followers["instagram"]
    except (KeyError, TypeError):
        existing_insta_followers = None
    if pod_social_links and existing_insta_followers is None:
        try:
            followers_count, recent_posts = get_pod_social_followers(pod_social_links)
            if isinstance(existing_followers, dict):
                pods.at[index, "followers_count"] = {
                    **existing_followers,  # Spread existing dict
                    **followers_count
                }
            else:
                pods.at[index, "followers_count"] = {
                    **followers_count
                }
            if recent_posts:
                pods.at[index, "recent_posts"] = recent_posts
        except Exception as e:
            print(pod_id," error in getting followers: ",e)

    youtube_id = pod["youtubeID"]
    existing_youtube_videos = pod["youtube_videos"]
    if youtube_id and existing_youtube_videos is None:
        try:
            latest_videos, subscribers = get_youtube_info_and_latest_videos(youtube_id, 5, 2)

            updated_latest_videos = []
            for video in latest_videos:
                video_id = video["youtubeVideoId"]
                transcript = video["transcript"]
                store_transcript(video_id, transcript)
                video["transcript"] = None
                updated_latest_videos.append(video)

            existing_followers = pods.at[index, "followers_count"]  # Changed from pod.get()

            if isinstance(existing_followers, dict):
                pods.at[index, "followers_count"] = {
                    **existing_followers,  # Spread existing dict
                    "youtube": subscribers
                }
            else:
                pods.at[index, "followers_count"] = {
                    "youtube": subscribers
                }
            pods.at[index, "youtube_videos"] = updated_latest_videos
        except Exception as e:
            print(pod_id," error in youtube videos : ",e)


In [ ]:

for index, pod in tqdm(pods.iterrows(), total=len(pods)):
# for index, pod in tqdm(pods.head(2).iterrows(), total=10):
    get_pod_all_info(pod, index)

  1%|          | 3/275 [01:02<1:37:27, 21.50s/it]

4thingswithamybrown  error in youtube videos :  'latestVideos'


  2%|▏         | 6/275 [02:49<2:33:38, 34.27s/it]

Datingintelligence  error in youtube videos :  'youtubeVideoId'


  3%|▎         | 7/275 [03:45<3:04:46, 41.37s/it]

...

wdwradio  error in youtube videos :  'youtubeVideoId'


 97%|█████████▋| 268/275 [2:15:34<03:47, 32.57s/it]

whatshouldireadnext  error in youtube videos :  'youtubeVideoId'


100%|██████████| 275/275 [2:19:58<00:00, 30.54s/it]

zengeen  error in youtube videos :  'youtubeVideoId'


In [205]:

pods.to_csv("1_pods_with_scrapped_data.csv", index=False)

In [5]:

pods = pd.read_csv("./1_pods_with_scrapped_data.csv")
pods.head(10)


,id,name,itunesID,image,youtubeID,categories,summary,spotifyId,socialLinks,category,apple_reviews,followers_count,recent_posts,youtube_videos
0,20vc,The Twenty Minute VC (20VC),9.582305e+08,https://static.libsyn.com/p/assets/d/2/b/a/d2b...,UCf0PBRjhf0rF8fWBIxTuoWA,"[""business"",""society_&_culture"",""technology""]",The Twenty Minute VC brings insightful intervi...,3j2KMcZTtgTNBKwtZBMHvl,"{""facebook"":""https://www.facebook.com/The Twen...",business,"{'average_rating': 4.4, 'rating_count': 479, '...","{'instagram': 11470, 'youtube': 160000}",{'instagram': [{'caption': 'Cos all the best a...,"[{'age_restricted': False, 'airDate': '2025072..."
1,402p,4.0 To Pro,1.672464e+09,https://megaphone.imgix.net/podcasts/073cc458-...,UCzxISXhBIzA-2wsnC7z9_Aw,"[""sports"",""education"",""leisure""]",4.0 to Pro is a pocket-sized podcast dedicated...,5xQf95A7d9UESlpZi2YGBZ,"{""applePodcasts"":""https://podcasts.apple.com/u...",sports,"{'average_rating': 4.9, 'rating_count': 339, '...","{'instagram': 3454, 'youtube': 1060}",{'instagram': [{'caption': '@4.0_to_pro Custom...,"[{'age_restricted': False, 'airDate': '2025071..."
2,4thingswithamybrown,4 Things with Amy Brown,1.442391e+09,https://www.omnycontent.com/d/playlist/e73c998...,UC7qHbziuYTsDYp9OxYXOJOA,"[""health&fitness"",""society&_culture"",""education""]",Join Amy Brown as she discusses 4 essential pr...,7qMQUCmvUFlqZ0J68anixa,"{""applePodcasts"":""https://podcasts.apple.com/u...",health&fitness,"{'average_rating': 4.9, 'rating_count': 5232, ...",{'instagram': 623540},{'instagram': [{'caption': 'only the best for ...,NaN
3,99invisible,99% Invisible,3.947753e+08,https://storage.googleapis.com/core-dev-361305...,UCVMF2HD4ZgC0QHpU9Yq5Xrw,"[""arts"",""society_&_culture"",""education""]","Hosted by Roman Mars, 99% Invisible explores t...",2VRS1IJCTn2Nlkg33ZVfkM,"{""facebook"":""https://www.facebook.com/99percen...",society_&_culture,"{'average_rating': 4.8, 'rating_count': 25366,...","{'instagram': 87855, 'youtube': 11400}",{'instagram': [{'caption': '99% Invisible invi...,"[{'age_restricted': False, 'airDate': '2025071..."
4,Crimeatorium,Crimeatorium True Crime,1.724979e+09,https://media.redcircle.com/images/2024/12/10/...,UCM_TCZ3r8wQGfol-C6CSKBw,"[""true_crime"",""society_&_culture"",""education""]",Join Madeline O'Brien as she delves into lesse...,3DsvYLaAUl1NV0eUrr6uI1,"{""facebook"":""https://www.facebook.com/Crimeato...",true_crime,"{'average_rating': None, 'rating_count': None,...","{'instagram': 0, 'youtube': 21900}",{'instagram': []},[]
5,Datingintelligence,Dating Intelligence the Podcast,1.513626e+09,https://static.libsyn.com/p/assets/2/3/f/1/23f...,UCcy0KXINyPCf6-lWEzFQTQg,"[""society_&_culture"",""health&fitness"",""educati...",Dating Intelligence the Podcast explores datin...,51I5oHDuLzecFM2VC2Elm1,"{""facebook"":null,""instagram"":""https://www.inst...",society_&_culture,"{'average_rating': 4.9, 'rating_count': 30, 'r...",{'instagram': 0},{'instagram': []},NaN
6,MidnightBurger,Midnight Burger,1.537653e+09,https://storage.googleapis.com/core-production...,UC4Hfu6qPbLI1Uecswv9Mm1g,"[""society_&_culture"",""fiction"",""leisure""]",Midnight Burger is a whimsical audio journey w...,3flVB3CeXIfVHwXjcKkNk8,"{""facebook"":""https://www.facebook.com/Midnight...",fiction,"{'average_rating': 4.9, 'rating_count': 2021, ...",{'instagram': 0},{'instagram': []},NaN
7,SPNthenandnow,Supernatural Then and Now,1.605385e+09,https://storage.googleapis.com/core-production...,UCHbjWmAa_7ixnzv1jlILv6A,"[""tv_&_film"",""society_&_culture"",""comedy""]",Supernatural Then and Now takes fans on a jour...,1c6D3QoBrXq9YezlQM1uer,"{""facebook"":null,""instagram"":""https://www.inst...",society_&_culture,"{'average_rating': 4.9, 'rating_count': 1991, ...","{'instagram': 0, 'youtube': 4130}",{'instagram': []},"[{'age_restricted': False, 'airDate': '2025071..."
8,Travel-in-10,Travel in 10: Travel Podcast,8.326049e+07,https://storage.googleapis.com/core-production...,UCLwR_buRFVTkb7yVWxc52-g,"[""leisure"",""s

In [26]:



# Get all pods with no YouTube videos
no_youtube_pods = pods[
    pods['youtube_videos'].isna() | 
    (pods['youtube_videos'].apply(lambda x: x is None or (isinstance(x, list) and len(x) == 0)))
]

print(f"Found {len(no_youtube_pods)} pods without YouTube videos")
print(f"Found {len(no_youtube_pods)} pods without YouTube videos")

youtubeids = no_youtube_pods['youtubeID'].tolist()

print(youtubeids)

for index, pod in tqdm(no_youtube_pods.iterrows(), total=len(no_youtube_pods)):
# for index, pod in tqdm(pods.head(2).iterrows(), total=10):
    get_pod_all_info(pod, index)


Found 31 pods without YouTube videos
Found 31 pods without YouTube videos
['UC7qHbziuYTsDYp9OxYXOJOA', 'UCcy0KXINyPCf6-lWEzFQTQg', 'UC4Hfu6qPbLI1Uecswv9Mm1g', 'UCoxza7rTVHE09GYLTh-QEOQ', 'UCP79TAt63VGJc-CheIm64aA', 'UCf5nlJFACZ2nUxOP87efMwA', 'UCmTcd3n8vAufGFV83fC87UA', 'UCEg_3EWYw7GdGr1m5Qu2QuQ', 'UC98f9qRr1AVxp1ucGGwiIkA', 'UCvPtZNWnrt70fKmqvoqW3OQ', 'UCmTcd3n8vAufGFV83fC87UA', 'UCLtREJY21xRfCuEKvdki1Kw', 'UCxoL7OXNn-JuGzZj7LQrtIA', 'UCvxWyn4rfcI2H9APhfUIB1Q', 'UC0A-v_DL-h76F75xik8h03Q', 'UCOPKJU_uSOH20DLRd1mSyNA', 'UCq_o0YLzQAaJDItIOa2T5xw', 'UC0o0Ge78Msm6HtACPXKESDQ', 'UCKhVag9OGIOJu_x430Dmedw', 'UCdDWGuTxkvpmD6tcZP2Gh9g', 'UCSvlnWkA909f87KjcaSZyPQ', 'UCT6QFE3peNry9PdO5uGj96g', 'UC-0xJNkg_MIeOs2XcDEFnbQ', 'UChFAxSGg6qtp9n7sZIoePHw', 'UCumRoq4stXAAVsSo5Js1_sg', 'UC83CJFLyDe72XgkKBd5a9IA', 'UCfcMA_uz6raLWfj-9S-uBoQ', 'UCXHiYKVUxYtaX151ezwaotQ', 'UCnp2k_FBdUOmwb05LDNv34w', 'UCleQU3Ro70D6z6o46wLt3tg', 'UCuKYdhaKACkpGeSzKfc4dtg']


100%|██████████| 31/31 [00:00<00:00, 20823.74it/s]


In [27]:
pods.to_csv("1_pods_with_scrapped_data.csv", index=False)